In [1]:
import pandas as pd
from pandas.io.json import json_normalize
from pandas import DataFrame
import os
from geopy import geocoders
from geopy.geocoders import GoogleV3
from config import gkey

import geopandas as gpd
from shapely import wkt
from shapely.wkt import loads
import re

In [2]:
# Pull in geojson address data from openaddresses
fname = "resources/us_il_city_of_evanston-addresses-city.geojson"

# read file to dataframe
gdf = gpd.read_file(fname)

simpledec = re.compile(r"\d*\.\d+")
def mround(match):
    return "{:.5f}".format(float(match.group()))

gdf.geometry = gdf.geometry.apply(lambda x: loads(re.sub(simpledec, mround, x.wkt)))
gdf

,id,unit,number,street,city,district,region,postcode,hash,geometry
0,,,811,CHICAGO AVE,EVANSTON,,IL,60202,0f1ae9d6dc3e261d,POINT (-87.67892 42.03261)
1,,,2730,CENTRAL ST,EVANSTON,,IL,60201,04d8292c6fe41f75,POINT (-87.71218 42.06420)
2,,,2726,CENTRAL ST,EVANSTON,,IL,60201,0f6aa09515f2553f,POINT (-87.71201 42.06420)
3,,,2720,CENTRAL ST,EVANSTON,,IL,60201,a9c1519f7bce1807,POINT (-87.71154 42.06419)
4,,,2722,CENTRAL ST,EVANSTON,,IL,60201,48834904e3e1b801,POINT (-87.71182 42.06420)
...,...,...,...,...,...,...,...,...,...,...
17130,,,1122,PITNER AVE,EVANSTON,,IL,60202,a06c3531387e9893,POINT (-87.70577 42.03833)
17131,,,1124,PITNER AVE,EVANSTON,,IL,60202,30eaae035c012e54,POINT (-87.70577 42.03839)
17132,,,1514,DEWEY AVE,EVANSTON,,IL,60201,c5f8058ef65e857f,POINT (-87.69672 42.04531)
17133,,,1141,DEWEY AVE,EVANSTON,,IL,60202,ba9fd44f5993932b,POINT (-87.69571 42.03897)


In [3]:
# Verify/Make pandas dataframe
addressdf = pd.DataFrame(gdf)

# Change 'geometry' column datatype from geometry to object
addressdf['geometry'] = addressdf.geometry.apply(lambda x: wkt.dumps(x, rounding_precision=5))


In [4]:
# Check dataframe
addressdf

,id,unit,number,street,city,district,region,postcode,hash,geometry
0,,,811,CHICAGO AVE,EVANSTON,,IL,60202,0f1ae9d6dc3e261d,POINT (-87.67892 42.03261)
1,,,2730,CENTRAL ST,EVANSTON,,IL,60201,04d8292c6fe41f75,POINT (-87.71218 42.06420)
2,,,2726,CENTRAL ST,EVANSTON,,IL,60201,0f6aa09515f2553f,POINT (-87.71201 42.06420)
3,,,2720,CENTRAL ST,EVANSTON,,IL,60201,a9c1519f7bce1807,POINT (-87.71154 42.06419)
4,,,2722,CENTRAL ST,EVANSTON,,IL,60201,48834904e3e1b801,POINT (-87.71182 42.06420)
...,...,...,...,...,...,...,...,...,...,...
17130,,,1122,PITNER AVE,EVANSTON,,IL,60202,a06c3531387e9893,POINT (-87.70577 42.03833)
17131,,,1124,PITNER AVE,EVANSTON,,IL,60202,30eaae035c012e54,POINT (-87.70577 42.03839)
17132,,,1514,DEWEY AVE,EVANSTON,,IL,60201,c5f8058ef65e857f,POINT (-87.69672 42.04531)
17133,,,1141,DEWEY AVE,EVANSTON,,IL,60202,ba9fd44f5993932b,POINT (-87.69571 42.03897)


In [5]:
# Combine address columns to one column
# addressdf['address'] = addressdf['street'].str.cat(addressdf['city'].str.cat(addressdf['region'],sep=" ")

# Remove extra strings from the latitude and longitude in the geometry column
addressdf['geometry'] = addressdf['geometry'].str.replace('POINT ', '').str.replace('(', '').str.replace(')', '')
addressdf

,id,unit,number,street,city,district,region,postcode,hash,geometry
0,,,811,CHICAGO AVE,EVANSTON,,IL,60202,0f1ae9d6dc3e261d,-87.67892 42.03261
1,,,2730,CENTRAL ST,EVANSTON,,IL,60201,04d8292c6fe41f75,-87.71218 42.06420
2,,,2726,CENTRAL ST,EVANSTON,,IL,60201,0f6aa09515f2553f,-87.71201 42.06420
3,,,2720,CENTRAL ST,EVANSTON,,IL,60201,a9c1519f7bce1807,-87.71154 42.06419
4,,,2722,CENTRAL ST,EVANSTON,,IL,60201,48834904e3e1b801,-87.71182 42.06420
...,...,...,...,...,...,...,...,...,...,...
17130,,,1122,PITNER AVE,EVANSTON,,IL,60202,a06c3531387e9893,-87.70577 42.03833
17131,,,1124,PITNER AVE,EVANSTON,,IL,60202,30eaae035c012e54,-87.70577 42.03839
17132,,,1514,DEWEY AVE,EVANSTON,,IL,60201,c5f8058ef65e857f,-87.69672 42.04531
17133,,,1141,DEWEY AVE,EVANSTON,,IL,60202,ba9fd44f5993932b,-87.69571 42.03897


In [6]:
# Remove un-needed columns
addressdf.drop(['unit', 'district'],axis=1,inplace=True)
addressdf

,id,number,street,city,region,postcode,hash,geometry
0,,811,CHICAGO AVE,EVANSTON,IL,60202,0f1ae9d6dc3e261d,-87.67892 42.03261
1,,2730,CENTRAL ST,EVANSTON,IL,60201,04d8292c6fe41f75,-87.71218 42.06420
2,,2726,CENTRAL ST,EVANSTON,IL,60201,0f6aa09515f2553f,-87.71201 42.06420
3,,2720,CENTRAL ST,EVANSTON,IL,60201,a9c1519f7bce1807,-87.71154 42.06419
4,,2722,CENTRAL ST,EVANSTON,IL,60201,48834904e3e1b801,-87.71182 42.06420
...,...,...,...,...,...,...,...,...
17130,,1122,PITNER AVE,EVANSTON,IL,60202,a06c3531387e9893,-87.70577 42.03833
17131,,1124,PITNER AVE,EVANSTON,IL,60202,30eaae035c012e54,-87.70577 42.03839
17132,,1514,DEWEY AVE,EVANSTON,IL,60201,c5f8058ef65e857f,-87.69672 42.04531
17133,,1141,DEWEY AVE,EVANSTON,IL,60202,ba9fd44f5993932b,-87.69571 42.03897


In [7]:

# fltraddress = addressdf[addressdf['addresses'].str.lower().str.contains('^6')]
# fltraddress

In [20]:
adds_sml = addressdf.sample(n = 100) 
adds_sml

,id,number,street,city,region,postcode,hash,geometry
7506,,3100,THAYER ST,EVANSTON,IL,60201,664afc5066fe79d4,-87.72096 42.06726
9885,,1581,DEWEY AVE,EVANSTON,IL,60201,d1941c5ab1bcd5b3,-87.69584 42.04680
161,,905,AUSTIN ST,EVANSTON,IL,60202,b5645980207af048,-87.68459 42.02567
2417,,2636,CRAWFORD AVE,EVANSTON,IL,60201,4277631a49c95cd5,-87.72840 42.06541
711,,601,LINDEN PL,EVANSTON,IL,60202,c36e478908828dac,-87.67977 42.02958
...,...,...,...,...,...,...,...,...
5254,,2204,MADISON PL,EVANSTON,IL,60202,cbb99ece58530e68,-87.70444 42.03084
11467,,1100,HARTREY AVE,EVANSTON,IL,60202,b416c14352d4a4f4,-87.70416 42.03758
3821,,2120,GREENWOOD ST,EVANSTON,IL,60201,f1b51a454dddeab9,-87.70329 42.04243
15826,,2322,NATHANIEL PL,EVANSTON,IL,60202,09b2eb1bd8e207a4,-87.70652 42.03626


In [21]:
# adds_sml.to_csv (r'static/data/sample/adds_sml.csv', index = True, header=True)

In [22]:
# adds_sml = pd.read_csv("static/data/sample/adds_sml.csv")
def format_coordinates(x):
    coordinates = x.split(' ')
    lat = str(coordinates[1])
    long = str(coordinates[0])
    formatted = lat + ', ' + long
    return formatted
adds_sml['geometry'] = adds_sml['geometry'].apply(format_coordinates)
adds_sml


,id,number,street,city,region,postcode,hash,geometry
7506,,3100,THAYER ST,EVANSTON,IL,60201,664afc5066fe79d4,"42.06726, -87.72096"
9885,,1581,DEWEY AVE,EVANSTON,IL,60201,d1941c5ab1bcd5b3,"42.04680, -87.69584"
161,,905,AUSTIN ST,EVANSTON,IL,60202,b5645980207af048,"42.02567, -87.68459"
2417,,2636,CRAWFORD AVE,EVANSTON,IL,60201,4277631a49c95cd5,"42.06541, -87.72840"
711,,601,LINDEN PL,EVANSTON,IL,60202,c36e478908828dac,"42.02958, -87.67977"
...,...,...,...,...,...,...,...,...
5254,,2204,MADISON PL,EVANSTON,IL,60202,cbb99ece58530e68,"42.03084, -87.70444"
11467,,1100,HARTREY AVE,EVANSTON,IL,60202,b416c14352d4a4f4,"42.03758, -87.70416"
3821,,2120,GREENWOOD ST,EVANSTON,IL,60201,f1b51a454dddeab9,"42.04243, -87.70329"
15826,,2322,NATHANIEL PL,EVANSTON,IL,60202,09b2eb1bd8e207a4,"42.03626, -87.70652"


In [23]:
# REPLACE 'space' with 'comma' in the geometry column
# adds_sml['geometry'] = adds_sml['geometry'].str.replace(' ',', ')
# adds_sml

In [24]:
# '''addresses = ["2620 N Drake Ave, Chicago, Illinois 60647",
#          "2622 N Drake Ave, Chicago, Illinois 60647",
#          "2626 N Drake Ave, Chicago, Illinois 60647",
#          "2630 N Drake Ave, Chicago, Illinois 60647",
#          "2648 N Drake Ave, Chicago, Illinois 60647"]'''

In [25]:
# '''df = DataFrame (addresses, columns=['Address'])
# df'''

In [26]:
# *** IF WE ONLY HAVE ADDRESSES AND NEED TO CONVERT TO LAT LONG ***

# API_KEY = os.getenv("API1234")
# g = GoogleV3(api_key=gkey)


# loc_coordinates = []
# loc_address = []
# for address in df.Address:
#     try:
#         inputAddress = address
#         location = g.geocode(inputAddress, timeout=15)
#         loc_coordinates.append((location.latitude, location.longitude))
#         loc_address.append(inputAddress)
#     except Exception as e:
#         print('Error, skipping address...', e)


# df_geocodes = pd.DataFrame({'coordinate':loc_coordinates,'address':loc_address})
# df_geocodes

In [27]:
# GRAB A SERIES OF STREET VIEW IMAGES FROM latlong PROVIDED
import json
# Import google_streetview for the api module
import google_streetview.api
import time
import glob


In [28]:
# create a params dict that will be updated with new city each iteration
params = {
    'size': '640x640', # max 640x640 pixels
    'heading': '90',
    'location': '',
    'pitch': '-0.76',
    'fov': 120,
    'key': gkey
}

# Loop through the cities_pd and run a lat/long search for each city
geometry_list = adds_sml['geometry'].to_list()
i = 0
for latlong in geometry_list:
    
    # update address key value
    params['location'] = latlong
    
    # Create a results object
#     results = google_streetview.api.results(params)
    results = google_streetview.api.results([params])
    
    
     

    # Download images to directory 'downloads'
    results.download_links(f"static/data/raw/geo{i}",)
    old_file_name = f"static/data/raw/geo{i}/gsv_0.jpg"
    new_file_name = f"static/data/raw/geo{i}/gsv_{i}.jpg"

    os.rename(old_file_name, new_file_name)
    
    #pull .jpeg into main folder rename .jpeg{i}
    
    i += 1
    
    # Wait for 1 second
    #time.sleep(1)

FileNotFoundError: [Errno 2] No such file or directory: 'static/data/raw/geo78/gsv_0.jpg' -> 'static/data/raw/geo78/gsv_78.jpg'

In [29]:
import shutil
# define the name of the directory to be created
path = "static/data/raw/locs"

try:
    os.mkdir(path)
except OSError:
    print ("Creation of the directory %s failed" % path)
else:
    print ("Successfully created the directory %s " % path)

Successfully created the directory static/data/raw/locs 


In [30]:
 
for root, dirs, files in os.walk('static/data/raw'):
    for file in files:
        if file.endswith('.jpg'):
            shutil.move(os.path.join(root, file),'static/data/raw/locs')

Error: Destination path 'static/data/raw/locs/gsv_1.jpg' already exists

In [19]:
import sys

!$sys.executable find .static/data/raw/locs -mindepth 1 ! -regex '^.static/data/raw/locs\(/.*\)?' -delete

/Users/datascience/opt/anaconda3/bin/python: can't open file 'find': [Errno 2] No such file or directory


In [ ]:
# # *** ORIGINAL PULL THAT WORKS ***
# # Define parameters for street view api
# params = [{
#     'size': '600x300', # max 640x640 pixels
#     'location': '41.9295582,-87.7159139',
#     'heading': '90',
#     'pitch': '-0.76',
#     'key': gkey
# }]

# # Create a results object
# results = google_streetview.api.results(params)

# # Download images to directory 'downloads'
# results.download_links('static/data/raw')

In [ ]:
## PRACTICING IMAGE PRE-PROCESSING

# # load and show an image with Pillow

# # load the image
# image = Image.open('static/data/test/brickornot.jpg')
# # summarize some details about the image
# print(image.format)
# print(image.mode)
# print(image.size)
# # show the image
# image.show()

In [ ]:
'''# load all images in a directory
loaded_images = list()
for filename in listdir('images'):
	# load image
	img_data = image.imread('images/' + filename)
	# store loaded image
	loaded_images.append(img_data)
	print('> loaded %s %s' % (filename, img_data.shape))'''


In [ ]:
%matplotlib inline
import PIL
from PIL import Image
from os import listdir
from matplotlib import image
from matplotlib import pyplot

# example of saving a grayscale version of a loaded image
from PIL import Image
# load the image
image = Image.open('static/data/raw/gsv_0.jpg')
# convert the image to grayscale
gs_image = image.convert(mode='L')
# report the size of the image
print(image.size)
# create a thumbnail and preserve aspect ratio
image.thumbnail((100,100))
# report the size of the thumbnail
print(image.size)
# save in jpeg format
gs_image.save('static/data/test_convert/gsv_0.jpg')

In [ ]:
# load the image again and show it
image2 = Image.open('static/data/test_convert/gsv_0.jpg')
# show the image
image2.show()

In [ ]:
'''# save as PNG format
image.save('opera_house.png', format='PNG')
# load the image again and inspect the format
image2 = Image.open('opera_house.png')
print(image2.format)'''